<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/test_app_web3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.2 MB/s eta 0:00:00


In [ ]:
from web3 import Web3

# --- 1. تنظیمات اولیه با Alchemy ---

# آدرس نود اتریوم از طریق Alchemy
# توجه: کلید API خود را جایگزین 'YOUR_ALCHEMY_API_KEY' کنید
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
web3 = Web3(Web3.HTTPProvider(alchemy_url))

# بررسی اتصال به شبکه
if not web3.is_connected():
    print("dont connect through alchemy")
    exit()
else:
    print("connected")


# --- 2. اطلاعات استخر ---

# آدرس قرارداد استخر مورد نظر در یونی‌سواپ V3
# برای مثال، از استخر USDC/WETH با کارمزد 0.05% استفاده شده است
pool_address = "0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082"


# --- 3. ABI قرارداد استخر ---

# ABI (Application Binary Interface) برای تعامل با توابع قرارداد هوشمند ضروری است.
# برای تابع tickSpacing()، تنها به بخش کوچکی از ABI کامل نیاز داریم.
pool_abi = [
    {
        "inputs": [],
        "name": "tickSpacing",
        "outputs": [
            {
                "internalType": "int24",
                "name": "",
                "type": "int24"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]


# --- 4. ایجاد نمونه قرارداد و فراخوانی تابع ---

# ایجاد یک نمونه از قرارداد استخر
pool_contract = web3.eth.contract(address=pool_address, abi=pool_abi)

try:
    # فراخوانی تابع 'tickSpacing' از قرارداد
    tick_spacing = pool_contract.functions.tickSpacing().call()

    # --- 5. نمایش نتیجه ---
    print("-" * 30)
    print(f"pool address: {pool_address}")
    print(f"pool Tick Spacing : {tick_spacing}")
    print("-" * 30)

except Exception as e:
    print(f"cause error : {e}")

connected
------------------------------
pool address: 0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082
pool Tick Spacing : 10
------------------------------


In [ ]:
import requests
import json

# --- 1. تنظیمات ---

# این URL، نقطه پایانی (Endpoint) API برای Subgraph یونی‌سواپ V3 است.
# این آدرس را از صفحه Subgraph در The Graph Explorer کپی کنید.
subgraph_url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

# آدرس استخری که می‌خواهیم اطلاعاتش را بگیریم
pool_id = "0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082"

# --- 2. کوئری GraphQL ---

# کوئری را که در Playground تست کردیم، در یک متغیر رشته‌ای چندخطی قرار می‌دهیم.
# می‌توانیم از متغیرهای پایتون برای داینامیک کردن کوئری استفاده کنیم.
query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    token0 {{
      symbol
    }}
    token1 {{
      symbol
    }}
    feeTier
    feesUSD
    totalValueLockedUSD
  }}
}}
"""

# --- 3. ارسال درخواست به Subgraph ---

try:
    # درخواست به صورت یک POST request با بدنه JSON ارسال می‌شود.
    response = requests.post(subgraph_url, json={'query': query})
    response.raise_for_status() # اگر خطایی (مثل 404 یا 500) رخ دهد، Exception ایجاد می‌کند.

    data = response.json()
    pool_data = data['data']['pool']

    # --- 4. نمایش نتایج ---

    # تبدیل مقادیر رشته‌ای به عددی برای فرمت‌دهی بهتر
    fees_usd = float(pool_data['feesUSD'])
    tvl_usd = float(pool_data['totalValueLockedUSD'])

    print("=" * 40)
    print(f"اطلاعات استخر: {pool_data['token0']['symbol']}/{pool_data['token1']['symbol']}")
    print(f"آدرس: {pool_data['id']}")
    print(f"نرخ کارمزد: {int(pool_data['feeTier']) / 10000}%")
    print("-" * 40)
    print(f"کل کارمزدهای تولید شده (USD): ${fees_usd:,.2f}")
    print(f"کل نقدینگی قفل شده (USD): ${tvl_usd:,.2f}")
    print("=" * 40)

except requests.exceptions.HTTPError as errh:
    print(f"Http Error: {errh}")
except requests.exceptions.ConnectionError as errc:
    print(f"Error Connecting: {errc}")
except requests.exceptions.Timeout as errt:
    print(f"Timeout Error: {errt}")
except requests.exceptions.RequestException as err:
    print(f"Oops: Something Else: {err}")
except KeyError:
    print("خطا: داده‌های استخر در پاسخ یافت نشد. ممکن است ID استخر اشتباه باشد.")

خطا: داده‌های استخر در پاسخ یافت نشد. ممکن است ID استخر اشتباه باشد.


In [ ]:
data

{'errors': [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]}

In [ ]:
import requests
import json

# --- 1. تنظیمات جدید ---

# کلید API خود را که از داشبورد The Graph دریافت کردید، اینجا قرار دهید.
# !!! هشدار: این کلید را محرمانه نگه دارید و در کدهای عمومی منتشر نکنید.
API_KEY = "YOUR_API_KEY_HERE"

# این URL، نقطه پایانی جدید برای Subgraph یونی‌سواپ V3 در شبکه اصلی اتریوم است.
# توجه کنید که کلید API شما به انتهای آدرس اضافه می‌شود.
api_key='server_b370ad407f52dc8fa6c0f902b8d7ea22'
subgraph_url = f"https://gateway.thegraph.com/api/{api_key}/subgraphs/id/ELUcS1a6e3Q3p64x8fFSN2Yg5bTfG3fra2j6eS23r8t"

# آدرس استخری که می‌خواهیم اطلاعاتش را بگیریم
pool_id = "0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082"


# --- 2. کوئری GraphQL (بدون تغییر) ---
query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    token0 {{
      symbol
    }}
    token1 {{
      symbol
    }}
    feeTier
    feesUSD
    totalValueLockedUSD
  }}
}}
"""


# --- 3. ارسال درخواست به Subgraph (بدون تغییر) ---

print("در حال ارسال درخواست به نقطه پایانی جدید The Graph...")

try:
    response = requests.post(subgraph_url, json={'query': query})
    response.raise_for_status()

    data = response.json()

    # بررسی اینکه آیا خود Subgraph خطایی برگردانده است یا خیر
    if 'errors' in data:
        print("خطا از سمت Subgraph:", data['errors'])
    else:
        pool_data = data['data']['pool']

        # --- 4. نمایش نتایج ---
        fees_usd = float(pool_data['feesUSD'])
        tvl_usd = float(pool_data['totalValueLockedUSD'])

        print("=" * 40)
        print(f"اطلاعات استخر: {pool_data['token0']['symbol']}/{pool_data['token1']['symbol']}")
        print(f"آدرس: {pool_data['id']}")
        print(f"نرخ کارمزد: {int(pool_data['feeTier']) / 10000}%")
        print("-" * 40)
        print(f"کل کارمزدهای تولید شده (USD): ${fees_usd:,.2f}")
        print(f"کل نقدینگی قفل شده (USD): ${tvl_usd:,.2f}")
        print("=" * 40)

except requests.exceptions.HTTPError as errh:
    print(f"Http Error: {errh}")
    print("متن پاسخ:", response.text) # برای دیدن جزئیات خطای HTTP
except requests.exceptions.RequestException as err:
    print(f"Request Error: {err}")
except KeyError:
    print("خطا: داده‌های استخر در پاسخ یافت نشد. ممکن است ID استخر اشتباه باشد یا کلید API نامعتبر باشد.")

در حال ارسال درخواست به نقطه پایانی جدید The Graph...
خطا از سمت Subgraph: [{'message': 'auth error: malformed API key'}]


In [4]:
import requests
import json

# --- 1. تنظیمات احراز هویت برای Ethereum Mainnet ---

# کلید API خود را که از تب "API Keys" در داشبورد The Graph Studio کپی کرده‌اید، اینجا قرار دهید.
# این کلید باید یک رشته ساده باشد (معمولاً با "tgk_" شروع می‌شود) و یک توکن JWT طولانی نیست.
#API_KEY = "server_b370ad407f52dc8fa6c0f902b8d7ea22"  # <--- کلید API صحیح خود را اینجا جایگذاری کنید
API_KEY='0f011cf8766c88189291aa0b95792828'
# شناسه زیرگراف رسمی Uniswap V3 برای شبکه اصلی اتریوم
#SUBGRAPH_ID = "ELUcS1a6e3Q3p64x8fFSN2Yg5bTfG3fra2j6eS23r8t"
SUBGRAPH_ID = "5zvR82Q90DdBoQfGwLi9Ge7Ja3LUg8DCTb554B6jnDs"

# =====>>> نکته کلیدی: این URL نقطه پایانی صحیح برای شبکه اصلی اتریوم است <<<=====
subgraph_url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID}"


# --- 2. اطلاعات کوئری (بدون تغییر) ---

# آدرس استخر USDC/WETH در شبکه اصلی اتریوم
pool_id = "0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082"

query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    token0 {{ symbol }}
    token1 {{ symbol }}
    feeTier
    feesUSD
  }}
}}
"""

# --- 3. ارسال درخواست ---

print(f"در حال ارسال کوئری به نقطه پایانی اتریوم Mainnet:")
print(subgraph_url)

try:
    response = requests.post(subgraph_url, json={'query': query})
    response.raise_for_status() # بررسی خطاهای HTTP مانند 401, 403, 404

    data = response.json()

    if 'errors' in data:
        print("\n!!! خطا از سمت The Graph دریافت شد:")
        print(json.dumps(data['errors'], indent=2))
    elif 'data' in data and data['data'].get('pool'):
        pool_data = data['data']['pool']
        fees_usd = float(pool_data['feesUSD'])

        print("\n" + "="*40)
        print("اطلاعات با موفقیت دریافت شد:")
        print(f"استخر: {pool_data['token0']['symbol']}/{pool_data['token1']['symbol']}")
        print(f"کل کارمزدهای تولید شده (USD): ${fees_usd:,.2f}")
        print("="*40)
    else:
        print("پاسخ معتبر بود، اما داده‌ای برای استخر مورد نظر یافت نشد.")
        print("پاسخ دریافتی:", data)

except requests.exceptions.HTTPError as e:
    print(f"\nخطای HTTP رخ داد: {e.response.status_code} {e.response.reason}")
    print("متن پاسخ سرور:", e.response.text)
except Exception as e:
    print(f"\nیک خطای غیرمنتظره رخ داد: {e}")

در حال ارسال کوئری به نقطه پایانی اتریوم Mainnet:
https://gateway.thegraph.com/api/0f011cf8766c88189291aa0b95792828/subgraphs/id/5zvR82Q90DdBoQfGwLi9Ge7Ja3LUg8DCTb554B6jnDs

!!! خطا از سمت The Graph دریافت شد:
[
  {
    "message": "subgraph not found: invalid subgraph ID: 5zvR82Q90DdBoQfGwLi9Ge7Ja3LUg8DCTb554B6jnDs"
  }
]


In [5]:
import requests
import json

subgraph_url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

pool_id = "0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082"

query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    token0 {{ symbol }}
    token1 {{ symbol }}
    feeTier
    feesUSD
  }}
}}
"""

response = requests.post(subgraph_url, json={'query': query})
data = response.json()

print(json.dumps(data, indent=2))


{
  "errors": [
    {
      "message": "This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com"
    }
  ]
}


In [6]:
GRAPH_API_KEY='0f011cf8766c88189291aa0b95792828'
pool_address='0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082'

In [8]:
import requests
import json

SUBGRAPH_ID = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

subgraph_url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    token0 {{ symbol }}
    token1 {{ symbol }}
    feeTier
    feesUSD
  }}
}}
"""

response = requests.post(subgraph_url, json={"query": query})
data = response.json()

print(json.dumps(data, indent=2, ensure_ascii=False))


{
  "data": {
    "pool": null
  }
}


In [9]:
response.status_code

200

In [10]:
data

{'data': {'pool': None}}

In [11]:
import requests
import json

API_KEY = "0f011cf8766c88189291aa0b95792828"
SUBGRAPH_ID = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

subgraph_url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

pool_id = "0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640"

query = f"""
{{
  pool(id: "{pool_id.lower()}") {{
    id
    token0 {{ symbol }}
    token1 {{ symbol }}
    feeTier
    feesUSD
  }}
}}
"""

response = requests.post(subgraph_url, json={"query": query})
print(json.dumps(response.json(), indent=2))


{
  "data": {
    "pool": {
      "feeTier": "500",
      "feesUSD": "287047400.8381036770727974181490179",
      "id": "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
      "token0": {
        "symbol": "USDC"
      },
      "token1": {
        "symbol": "WETH"
      }
    }
  }
}


In [12]:
import requests, json

#API_KEY = "0f011cf8766c88189291aa0b95792828"
SUBGRAPH_ID = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

pool_id = "0x5ae13baaef0620fdae1d355495dc51a17adb4082"   # lowercase

query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    token0 {{ symbol }}
    token1 {{ symbol }}
    feeTier
    liquidity
    volumeUSD
    feesUSD
  }}
}}
"""

r = requests.post(url, json={"query": query})
print(json.dumps(r.json(), indent=2))


{
  "data": {
    "pool": {
      "feeTier": "500",
      "feesUSD": "492832.3145213585",
      "id": "0x5ae13baaef0620fdae1d355495dc51a17adb4082",
      "liquidity": "2234668732470527189",
      "token0": {
        "symbol": "PAXG"
      },
      "token1": {
        "symbol": "USDC"
      },
      "volumeUSD": "985664629.042717"
    }
  }
}


In [ ]:
import requests, json

API_KEY = "0f011cf8766c88189291aa0b95792828"
SUBGRAPH_ID = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

pool_id = "0x5ae13baaef0620fdae1d355495dc51a17adb4082"   # lowercase

query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    token0 {{ symbol }}
    token1 {{ symbol }}
    feeTier
    liquidity
    volumeUSD
    feesUSD
  }}
}}
"""

r = requests.post(url, json={"query": query})
print(json.dumps(r.json(), indent=2))

In [4]:
import requests, json

API_KEY = "0f011cf8766c88189291aa0b95792828"
SUBGRAPH_ID = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

pool_id = "0x5ae13baaef0620fdae1d355495dc51a17adb4082"   # lowercase

query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    token0 {{ symbol }}
    token1 {{ symbol }}
    feeTier
    liquidity
    volumeUSD
    feesUSD
    totalValueLockedUSD
    createdAtTimestamp
  }}
}}
"""

r = requests.post(url, json={"query": query})
print(json.dumps(r.json(), indent=2, ensure_ascii=False))


{
  "data": {
    "pool": {
      "createdAtTimestamp": "1655264999",
      "feeTier": "500",
      "feesUSD": "493032.042061845",
      "id": "0x5ae13baaef0620fdae1d355495dc51a17adb4082",
      "liquidity": "2235669318280199032",
      "token0": {
        "symbol": "PAXG"
      },
      "token1": {
        "symbol": "USDC"
      },
      "totalValueLockedUSD": "11259690.59809312034120647625671137",
      "volumeUSD": "986064084.12369"
    }
  }
}


In [7]:
data=r.json()
data

{'data': {'pool': {'createdAtTimestamp': '1655264999',
   'feeTier': '500',
   'feesUSD': '493032.042061845',
   'id': '0x5ae13baaef0620fdae1d355495dc51a17adb4082',
   'liquidity': '2235669318280199032',
   'token0': {'symbol': 'PAXG'},
   'token1': {'symbol': 'USDC'},
   'totalValueLockedUSD': '11259690.59809312034120647625671137',
   'volumeUSD': '986064084.12369'}}}

In [9]:
def find_key(data, target_key):
    results = []
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                results.append(value)
            else:
                results.extend(find_key(value, target_key))
    elif isinstance(data, list):
        for item in data:
            results.extend(find_key(item, target_key))
    return results

In [21]:
start_dt=int(find_key(data,'createdAtTimestamp')[0])
start_dt

1655264999

In [22]:
import time

In [23]:
time.time()

1763756237.7008653

In [27]:
days=(int(time.time())-start_dt)/(24*60*60)
days

1255.687997685185

In [25]:
total_fee=float(find_key(data,'feesUSD')[0])
total_fee

493032.042061845

In [26]:
tvl=float(find_key(data,'totalValueLockedUSD')[0])
tvl

11259690.59809312

In [28]:
total_fee/days

392.6389700074633

In [30]:
((total_fee/days)/tvl)*365

0.012727989530813116